# Исследование надежности заемщиков


## Загрузка таблицы и изучение общей информации

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Выявление и обработка пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставляю их как есть. Тем более этот столбец не понадобится для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Выявление и обработка пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ данных

### Исследование зависимости кол-ва детей заемщика и возвратом кредита в срок

In [30]:
data_grouped1 = data.groupby(['children']).agg({'debt':'sum', 'children':'count'})
data_grouped1['persent']=data_grouped1['debt']/data_grouped1['children']

#print(data_grouped1)

data_zero=100*data_grouped1.loc[0, 'persent']
data_one=100*data_grouped1.loc[1, 'persent']
data_two=100*data_grouped1.loc[2, 'persent']
data_three=100*data_grouped1.loc[3, 'persent']
data_four=100*data_grouped1.loc[4, 'persent']
data_five=data_grouped1.loc[5, 'persent']

print(f'Для заемщиков без детей, просрочка составила {data_zero:.2f}%')
print(f'Для заемщиков с одним ребенком, просрочка составила {data_one:.2f}%')
print(f'Для заемщиков с двумя детьми, просрочка составила {data_two:.2f}%')
print(f'Для заемщиков с тремя детьми, просрочка составила {data_three:.2f}%')
print(f'Для заемщиков с четыремя детьми, просрочка составила {data_four:.2f}%')
print(f'Для заемщиков пятью детьми, просрочка составила {data_five:.2f}%')

Для заемщиков без детей, просрочка составила 7.54%
Для заемщиков с одним ребенком, просрочка составила 9.23%
Для заемщиков с двумя детьми, просрочка составила 9.45%
Для заемщиков с тремя детьми, просрочка составила 8.18%
Для заемщиков с четыремя детьми, просрочка составила 9.76%
Для заемщиков пятью детьми, просрочка составила 0.00%


**Вывод:** Зависимость между количеством детей у заемщика и возвратом кредита в срок - есть.
У заемщиков без детей наименьший процент просрочек по кредиту. В целом логично, наличие детей налагает на семейный бюджет дополнительную нагрузку, что в перспективе может помешать своевременной оплате кредита.

### Исследование зависимости статуса семейного положения и возвратом кредита в срок

In [41]:
data_grouped2 = data.groupby(['family_status']).agg({'debt':'sum', 'family_status':'count'})
data_grouped2['persent']=data_grouped2['debt']/data_grouped2['family_status']

print(data_grouped2)

#data_2=100*data_grouped2.loc[0, 'persent']
#print(f'Для не женатых заемщиков, просрочка составила {data_2:.2f}%')  - НЕ ПОНИМАЮ ПОЧЕМУ НЕ РАБОТАЕТ. Не работает такой код вывода и во всех дальнейших задачах

                       debt  family_status   persent
family_status                                       
Не женат / не замужем   273           2796  0.097639
в разводе                84           1189  0.070648
вдовец / вдова           63            951  0.066246
гражданский брак        385           4146  0.092861
женат / замужем         927          12266  0.075575


**Вывод:** У заемщиков находящихся в гражданском браке и незамужних наибольший процент просрочек по кредиту ~10%. Воможно, этим двум сегментам нужно ужесточить условия выдачи кредита.

### Исследование зависимости уровня дохода и возвратом кредита в срок

In [36]:
#A > 1.000.000
#B <200.000
#C >50.000
#D <50.000
#E <30.000

data_grouped3 = data.groupby(['total_income_category']).agg({'debt':'sum', 'total_income_category':'count'})
data_grouped3['persent']=data_grouped3['debt']/data_grouped3['total_income_category']

print(data_grouped3)

                       debt  total_income_category   persent
total_income_category                                       
A                         2                     25  0.080000
B                       354                   5014  0.070602
C                      1353                  15938  0.084891
D                        21                    349  0.060172
E                         2                     22  0.090909


**Вывод:** Выборка заемщиков с низким и высоким уровнем дохода слишком мала, для объективного вывода. Самый массовый заемщик - имеет доход от 50.000 до 200.000, при этом имеет уровень задолженности 8,5%. Глобального вывода по уровню дохода не могу дать.

### Исследование зависимости целей кредита и его возвратом в срок

In [34]:
data_grouped4 = data.groupby(['purpose_category']).agg({'debt':'sum', 'purpose_category':'count'})
data_grouped4['persent']=data_grouped4['debt']/data_grouped4['purpose_category']

print(data_grouped4)

                          debt  purpose_category   persent
purpose_category                                          
операции с автомобилем     400              4281  0.093436
операции с недвижимостью   780             10754  0.072531
получение образования      369              3989  0.092504
проведение свадьбы         183              2324  0.078744


**Вывод:** У заемщиков с целью кредита "операции с недвижимостью" наименьший процент просрочки. Если цели кредита "получение образования" или "операции с автомобилем" шанс просрочки самый большой.

## Общий вывод.

В  ходе исследования был выявлен идеальный портрет заемщика это: 
- женатая пара, 
- без детей, 
- уровень дохода от 200тыс и 
- цель кредита "операции с недвижимостью". 

У этих категорий заемщиков наименьший процент просрочек по кредитам. Возможно, стоит разработать специальное предложение под эту категорию заемщиков. 
И наоборот: заемщики не в браке или находящиеся в гражданском браке, с 4мя детьми, имеющие доход меньше 50тыс и целью кредита "покупка автомоюиля" - самые потенциально рискованные заемщики. Возможно стоит ужесточить правила выдачи кредита для заемщиков этой категории.

Для ускорения дальнейших отчетов для отдела разработки необходимо убрать возможность потенциальному заемщику самостоятельно писать цель кредита (исключает ошибки и опечатки со стороны заемщика). Сделать выпадающий список, чтоб потенциальный заемщик выбрал нужную категорию. Необходимо сделать графы "доход" и "стаж" (total_income / days_employed) обязательными для заполнения.  Данные графы важные критерии как для анализа так и для принятия решения о возможности одобрения заявки на кредит.